**Repository**: https://github.com/EstebanMqz/SP500-Risk-Optimized-Portfolios-PostCovid-ML

### <font color= 'orange'> **S & P 500 Risk Optimized Portfolios PostCovid ML (14 Apr $yr_{(2020-2023)}$**<font> 

In [2]:
#Import dependencies
import functions as fn
import data as dt
import visualizations as vs

#Libraries
import numpy as np
import pandas as pd
import matplotlib as plt

import scipy
import scipy.stats as st
from scipy import optimize

import sklearn
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

from yahoofinancials import YahooFinancials 
from tabulate import tabulate
import IPython.display as d

import datetime 
import time

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning)

##### <span style='color:red'> Project Creators:</span> Create requirements.txt file.<br>

In [ ]:
docstring = """
# -- --------------------------------------------------------------------------------------------------  -- # 
# -- project: S&P500-Risk-Optimized-Portfolios-PostCovid-ML                                              -- # 
# -- script: requirements.txt: txt file to download Python modules for execution                         -- # 
# -- author: EstebanMqz                                                                                  -- # 
# -- license: CC BY 3.0                                                                                  -- # 
# -- repository: SP500-Risk-Optimized-Portfolios-PostCovid-ML/blob/main/requirements.txt                 -- #                                  
# -- --------------------------------------------------------------------------------------------------  -- # 
\n
"""

fn.get_requirements(docstring)

##### <span style='color:green'> Project Users:</span> Install libraries in requirements.txt file.<br>

In [ ]:
dt.library_install("requirements.txt")

Yahoo Finance data fetching is limited and other data sources like quandl (with an API key) or MT5 for Criptos/Forex can be used. <br>
Fetching data from Yahoo Finance by batches in different cells could be a solution, these the tickers from A-Z: <br> 

In [28]:
tickers=fn.SP500_tickers(50)
tickers[0][:5], tickers[-1][-5:]

(['MMM', 'AOS', 'ABT', 'ABBV', 'ACN'], ['ZBH', 'ZION', 'ZTS'])

In [5]:
data = pd.concat([dt.get_historical_price_data(tickers[0][i], 3) for i in range (0, len(tickers[0]))], axis=1)


In [27]:
tickers[1] = [i.replace(".", "-") for i in tickers[1]]
data1 = pd.concat([dt.get_historical_price_data(tickers[1][i], 3) for i in range (0, len(tickers[1]))], axis=1)

In [9]:
data2 = pd.concat([dt.get_historical_price_data(tickers[2][i], 3) for i in range (0, len(tickers[2]))], axis=1)


In [12]:
data3 = pd.concat([dt.get_historical_price_data(tickers[3][i], 3) for i in range (0, len(tickers[3]))], axis=1)

In [13]:
data4 = pd.concat([dt.get_historical_price_data(tickers[4][i], 3) for i in range (0, len(tickers[4]))], axis=1)

In [14]:
data5 = pd.concat([dt.get_historical_price_data(tickers[5][i], 3) for i in range (0, len(tickers[5]))], axis=1)

In [15]:
data6 = pd.concat([dt.get_historical_price_data(tickers[6][i], 3) for i in range (0, len(tickers[6]))], axis=1)

In [16]:
data7 = pd.concat([dt.get_historical_price_data(tickers[7][i], 3) for i in range (0, len(tickers[7]))], axis=1)

In [17]:
data8 = pd.concat([dt.get_historical_price_data(tickers[8][i], 3) for i in range (0, len(tickers[8]))], axis=1)

In [18]:
data9 = pd.concat([dt.get_historical_price_data(tickers[9][i], 3) for i in range (0, len(tickers[9]))], axis=1)

In [44]:
#concatenate all dataframes
sp500 = pd.concat([data, data1, data2, data3, data4, data5, data6, data7, data8, data9], axis=1)
sp500.to_csv("Data/sp500.csv")

In [45]:
SP = pd.read_csv("Data/sp500.csv", index_col=0) 
SP.head()

,MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,ADP,AAP,...,WY,WHR,WMB,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA
formatted_date,,,,,,,,,,,,,,,,,,,,,
2020-05-01,132.659531,39.439060,85.580452,72.788933,172.786255,64.027557,32.627090,343.839996,132.085907,108.905914,...,15.830580,101.116699,15.405546,175.907654,256.132141,80.830002,57.244247,66.869682,79.337418,225.550003
2020-05-04,132.480927,39.751537,85.561401,71.927826,172.248978,65.986374,32.292645,349.109985,135.904938,112.193924,...,15.663200,95.555687,15.887228,176.099503,260.399994,80.449997,57.437580,66.155525,78.655838,228.770004
2020-05-05,131.614990,39.921986,88.609337,75.011963,174.254196,67.796799,32.478447,356.130005,137.165405,113.202744,...,15.434158,98.475899,15.779264,182.163300,267.184631,79.669998,58.256935,62.777779,78.835701,233.149994
2020-05-06,130.516953,41.332893,86.751999,75.055893,173.064514,72.090363,31.958202,362.519989,135.980194,113.174721,...,16.059629,96.556137,15.455378,176.195465,268.419067,79.250000,56.360458,60.857288,78.731560,232.419998
2020-05-07,130.106323,41.010944,89.523697,74.001495,179.521576,72.317894,32.292645,366.779999,138.999695,111.726883,...,16.412003,96.348839,15.953670,185.319931,269.002777,79.599998,55.200470,61.330170,80.085258,233.860001


In [46]:
SP.tail()

,MMM,AOS,ABT,ABBV,ACN,ATVI,ADM,ADBE,ADP,AAP,...,WY,WHR,WMB,WTW,GWW,WYNN,XEL,XYL,YUM,ZBRA
formatted_date,,,,,,,,,,,,,,,,,,,,,
2023-04-24,105.059998,68.613686,110.400002,164.080002,277.250000,86.089996,80.800003,377.339996,215.470001,126.830002,...,30.600000,140.699997,30.330000,240.259995,675.320007,114.480003,71.029999,104.540001,138.899994,288.190002
2023-04-25,104.370003,68.265244,109.970001,164.899994,270.700012,86.739998,75.900002,369.589996,211.690002,123.779999,...,29.889999,132.550003,29.930000,239.589996,671.830017,111.629997,71.180000,102.919998,137.809998,284.100006
2023-04-26,102.919998,67.200005,108.750000,161.800003,271.209991,76.809998,77.029999,363.059998,210.830002,122.330002,...,29.190001,129.020004,29.440001,237.770004,659.820007,109.449997,69.680000,100.980003,138.009995,280.420013
2023-04-27,105.330002,68.559998,109.500000,148.869995,275.450012,77.610001,78.010002,371.420013,216.160004,124.900002,...,29.389999,135.389999,29.809999,225.809998,696.359985,110.220001,70.260002,103.830002,139.800003,284.119995
2023-04-28,106.220001,68.290001,110.470001,151.119995,280.290009,77.709999,78.080002,377.559998,220.000000,125.529999,...,29.910000,139.589996,30.260000,231.600006,695.570007,114.279999,69.910004,103.839996,140.580002,288.029999
